# 5. Game map analysis

## Preparation

In [ ]:
%run "../Functions/4. User comparison.ipynb"
print("5. Game map analysis")

In [ ]:
import matplotlib.image as image

## Functions

In [ ]:
# eg _types = ['death', 'reach', 'add', 'craft', 'remove', 'select', 'selectmenu']

def filterAndLocateEvents(_types, _rmDF):
    _mapRelevantColumns = ['type', QUserId, 'section', 'coordinates']
    _locatedEventsDF = _rmDF.loc[:,_mapRelevantColumns].dropna()

    # types
    _locatedEventsDF = _locatedEventsDF[_locatedEventsDF['type'].isin(_types)]

    # section
    _locatedEventsDF = _locatedEventsDF[(_locatedEventsDF['section'].str.startswith(tutorialStem))]

    # x,y
    _locatedEventsDF['x'] = _locatedEventsDF['coordinates']
    _locatedEventsDF['y'] = _locatedEventsDF['coordinates']

    _f = FloatProgress(min=0, max=len(_locatedEventsDF.index))
    display(_f)

    for index in _locatedEventsDF.index:
        _coordinates = re.findall('-*\d+', _locatedEventsDF.loc[index,'coordinates'])
        _locatedEventsDF.loc[index, 'x'] = int(_coordinates[0])
        _locatedEventsDF.loc[index, 'y'] = int(_coordinates[1])
        _f.value += 1

    _locatedEventsDF = _locatedEventsDF.drop('coordinates', 1)

    _f.close()
    del _f
    
    return _locatedEventsDF

In [ ]:
def getConstants():
    _gameStartCoordinates = [-229,-608]
    
    # from RedMetrics
    #_gameEndCoordinates = [441,-472]
    
    # after tinkering
    _gameEndCoordinates = [450,-475]
    

    # only for '../../images/map.tutorial1.png'
    _imgStartCoordinates = [833,432]
    _imgEndCoordinates = [1333,334]

    _scaleX = (_gameStartCoordinates[0]-_gameEndCoordinates[0])/(_imgStartCoordinates[0]-_imgEndCoordinates[0])
    _scaleY = (_gameStartCoordinates[1]-_gameEndCoordinates[1])/(_imgStartCoordinates[1]-_imgEndCoordinates[1])
    
    _offsetX = _gameStartCoordinates[0] - (_imgStartCoordinates[0]*_scaleX)
    _offsetY = _gameStartCoordinates[1] - (_imgStartCoordinates[1]*_scaleY)
    
    _constants = pd.Series([_scaleX,_scaleY,_offsetX,_offsetY], index=['scaleX', 'scaleY', 'offsetX', 'offsetY'])
    
    return _constants
    
def getGraphPosition(_imgPos, _constants):
    _x = getXGraphPosition(_imgPos[0], _constants)
    _y = getYGraphPosition(_imgPos[1], _constants)
    return [_x, _y]

def getXGraphPosition(_xImg, _constants):
    return _constants.scaleX*_xImg + _constants.offsetX

def getYGraphPosition(_yImg, _constants):
    return _constants.scaleY*_yImg + _constants.offsetY

In [ ]:
def plotLocatedEvents(_XYEventsDF, _types, figsize=(18,12), _kdeplot = False,
    _kdebw='scott',_kdecut=3,_kdeshade=False,_kdeshade_lowest=False,_kdecbar=False,_kdealpha=0.5,
                     ):
    
    _constants = getConstants()

    fig = plt.figure(figsize=figsize)

    # background map image display
    _mapBackground = image.imread('../../images/map.tutorial1.png')

    # background image dimensions
    _minX = 0
    _maxX = _mapBackground.shape[1]
    _minY = _mapBackground.shape[0]
    _maxY = 0

    plt.imshow(_mapBackground, aspect='auto', zorder=-1, \
               extent=(getXGraphPosition(_minX,_constants), \
                       getXGraphPosition(_maxX,_constants), \
                       getYGraphPosition(_minY,_constants), \
                       getYGraphPosition(_maxY,_constants)), \
               alpha=0.5)


    _colors = ['blue', 'red', 'green', 'yellow', 'black', 'cyan', 'orange', 'purple', 'brown', 'pink', 'olive', 'cyan', ]
    _alphas = [ 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, ]
    _scatters = {}

    for i, t in enumerate(_types):
        _pts = _XYEventsDF[_XYEventsDF['type'] == t]
        x = _pts['x']
        y = _pts['y']
        
        j = min(i, min(len(_alphas), len(_colors))-1)
        _scatters[t] = plt.plot(x, y, 'o', c=_colors[j], lw=0, alpha=_alphas[j])
        
    if _kdeplot:
        ax = sns.kdeplot(_XYEventsDF['x'],_XYEventsDF['y'],
                         bw=_kdebw,
                         cut=_kdecut,
                         shade=_kdeshade,
                         shade_lowest=_kdeshade_lowest,
                         cbar=_kdecbar,
                         alpha=_kdealpha,
                        )

    plt.legend([ x[0] for x in _scatters.values()], list(_scatters.keys()))

    # graph dimensions and extrema on tutorial1
    # _minX=-1152
    # _maxX=539
    # _minY=-1161
    # _maxY=-100
    plt.xlim([-1200, 600])
    plt.ylim([-1220, -20])
    
    plt.show()
    return fig